In [5]:
import torch
import torch.nn as nn

In [6]:
model = nn.Linear(1, 2)

In [7]:
model

Linear(in_features=1, out_features=2, bias=True)

In [8]:
net = nn.DataParallel(model)

In [9]:
net

DataParallel(
  (module): Linear(in_features=1, out_features=2, bias=True)
)

In [11]:
for name, p in net.module.named_parameters():
    print(name)

weight
bias


In [2]:
hidden_size = 512
attn_linear = nn.Sequential(
    nn.Linear(hidden_size, hidden_size),
    nn.Tanh(),
    nn.Linear(hidden_size, 1),
    #nn.Softmax()
)

In [7]:
x = 0.01
for i in range(2, 20, 2):
    x = x * 0.5
    print(i, x)

2 0.005
4 0.0025
6 0.00125
8 0.000625
10 0.0003125
12 0.00015625
14 7.8125e-05
16 3.90625e-05
18 1.953125e-05


In [2]:
import torch
import numpy as np

In [1]:
import torch
import numpy as np

word2vec = {}
idx = 0

with open('/home/quanguet/datasets/glove/glove.840B.300d.txt', 'rb') as f:
    for line in f:
        line = line.decode().split()
        word = line[0]
        
        if not line[1].replace('.','',1).replace('e-', '').replace('-', '', 1).isdigit():
            print('len=', len(line[1:]), line[1:5])
            continue
        vec = torch.tensor(np.asarray(line[1:], dtype=np.float64))
        word2vec[word] = vec
        idx += 1

len= 302 ['.', '.', '-0.1573', '-0.29517']
len= 301 ['name@domain.com', '0.0061218', '0.39595', '-0.22079']
len= 304 ['.', '.', '.', '.']
len= 301 ['name@domain.com', '0.33865', '0.12698', '-0.16885']
len= 301 ['.', '0.035974', '-0.024421', '0.71402']
len= 303 ['.', '.', '.', '0.033459']
len= 301 ['name@domain.com', '0.33529', '0.32949', '0.2646']
len= 301 ['name@domain.com', '0.48374', '0.49669', '-0.25089']


KeyboardInterrupt: 

In [3]:
run configs/lf_disc_config.py

ERROR:root:File `'configs/lf_disc_config.py'` not found.


In [4]:
"""
A Vocabulary maintains a mapping between words and corresponding unique
integers, holds special integers (tokens) for indicating start and end of
sequence, and offers functionality to map out-of-vocabulary words to the
corresponding token.
"""
import json
import os
from typing import List


class Vocabulary(object):
    """
    A simple Vocabulary class which maintains a mapping between words and
    integer tokens. Can be initialized either by word counts from the VisDial
    v1.0 train dataset, or a pre-saved vocabulary mapping.
    Parameters
    ----------
    word_counts_path: str
        Path to a json file containing counts of each word across captions,
        questions and answers of the VisDial v1.0 train dataset.
    min_count : int, optional (default=0)
        When initializing the vocabulary from word counts, you can specify a
        minimum count, and every token with a count less than this will be
        excluded from vocabulary.
    """

    PAD_TOKEN = "<PAD>"
    SOS_TOKEN = "<S>"
    EOS_TOKEN = "</S>"
    UNK_TOKEN = "<UNK>"

    PAD_INDEX = 0
    SOS_INDEX = 1
    EOS_INDEX = 2
    UNK_INDEX = 3

    def __init__(self, word_counts_path: str, min_count: int = 5):
        if not os.path.exists(word_counts_path):
            raise FileNotFoundError(
                f"Word counts do not exist at {word_counts_path}"
            )

        with open(word_counts_path, "r") as word_counts_file:
            word_counts = json.load(word_counts_file)

            # form a list of (word, count) tuples and apply min_count threshold
            word_counts = [
                (word, count)
                for word, count in word_counts.items()
                if count >= min_count
            ]
            # sort in descending order of word counts
            word_counts = sorted(word_counts, key=lambda wc: -wc[1])
            words = [w[0] for w in word_counts]

        self.word2index = {}
        self.word2index[self.PAD_TOKEN] = self.PAD_INDEX
        self.word2index[self.SOS_TOKEN] = self.SOS_INDEX
        self.word2index[self.EOS_TOKEN] = self.EOS_INDEX
        self.word2index[self.UNK_TOKEN] = self.UNK_INDEX
        for index, word in enumerate(words):
            self.word2index[word] = index + 4

        self.index2word = {
            index: word for word, index in self.word2index.items()
        }

    @classmethod
    def from_saved(cls, saved_vocabulary_path: str) -> "Vocabulary":
        """Build the vocabulary from a json file saved by ``save`` method.
        Parameters
        ----------
        saved_vocabulary_path : str
            Path to a json file containing word to integer mappings
            (saved vocabulary).
        """
        with open(saved_vocabulary_path, "r") as saved_vocabulary_file:
            cls.word2index = json.load(saved_vocabulary_file)
        cls.index2word = {
            index: word for word, index in cls.word2index.items()
        }

    def to_indices(self, words: List[str]) -> List[int]:
        return [self.word2index.get(word, self.UNK_INDEX) for word in words]

    def to_words(self, indices: List[int]) -> List[str]:
        return [
            self.index2word.get(index, self.UNK_TOKEN) for index in indices
        ]

    def save(self, save_vocabulary_path: str) -> None:
        with open(save_vocabulary_path, "w") as save_vocabulary_file:
            json.dump(self.word2index, save_vocabulary_file)

    def __len__(self):
        return len(self.index2word)

In [5]:
vocab = Vocabulary('/home/quanguet/datasets/visdial/visdial_1.0_word_counts_train.json')

In [6]:
len(vocab)

11322

In [7]:
len(vocab.word2index)

11322

In [9]:
word2id = vocab.word2index

In [10]:
word2id

{'<PAD>': 0,
 '<S>': 1,
 '</S>': 2,
 '<UNK>': 3,
 '?': 4,
 'is': 5,
 'the': 6,
 'a': 7,
 'no': 8,
 'are': 9,
 'yes': 10,
 'it': 11,
 'there': 12,
 'what': 13,
 'see': 14,
 'in': 15,
 'you': 16,
 'any': 17,
 'on': 18,
 'color': 19,
 'of': 20,
 ',': 21,
 'can': 22,
 'and': 23,
 'i': 24,
 'people': 25,
 'they': 26,
 'do': 27,
 'or': 28,
 'does': 29,
 'white': 30,
 'how': 31,
 "'s": 32,
 'he': 33,
 "n't": 34,
 'to': 35,
 'this': 36,
 'with': 37,
 'like': 38,
 'other': 39,
 'wearing': 40,
 'tell': 41,
 '1': 42,
 'not': 43,
 'ca': 44,
 'man': 45,
 'look': 46,
 'black': 47,
 'many': 48,
 'have': 49,
 'looks': 50,
 'trees': 51,
 'table': 52,
 'photo': 53,
 '2': 54,
 'old': 55,
 'sunny': 56,
 'at': 57,
 'she': 58,
 'picture': 59,
 'be': 60,
 'but': 61,
 'his': 62,
 'brown': 63,
 'just': 64,
 'day': 65,
 'that': 66,
 'blue': 67,
 'an': 68,
 'think': 69,
 'around': 70,
 'red': 71,
 'woman': 72,
 'sitting': 73,
 'room': 74,
 'kind': 75,
 'visible': 76,
 'some': 77,
 'water': 78,
 'person': 79,
 'u

In [11]:
id2word = vocab.index2word

In [12]:
id2word

{0: '<PAD>',
 1: '<S>',
 2: '</S>',
 3: '<UNK>',
 4: '?',
 5: 'is',
 6: 'the',
 7: 'a',
 8: 'no',
 9: 'are',
 10: 'yes',
 11: 'it',
 12: 'there',
 13: 'what',
 14: 'see',
 15: 'in',
 16: 'you',
 17: 'any',
 18: 'on',
 19: 'color',
 20: 'of',
 21: ',',
 22: 'can',
 23: 'and',
 24: 'i',
 25: 'people',
 26: 'they',
 27: 'do',
 28: 'or',
 29: 'does',
 30: 'white',
 31: 'how',
 32: "'s",
 33: 'he',
 34: "n't",
 35: 'to',
 36: 'this',
 37: 'with',
 38: 'like',
 39: 'other',
 40: 'wearing',
 41: 'tell',
 42: '1',
 43: 'not',
 44: 'ca',
 45: 'man',
 46: 'look',
 47: 'black',
 48: 'many',
 49: 'have',
 50: 'looks',
 51: 'trees',
 52: 'table',
 53: 'photo',
 54: '2',
 55: 'old',
 56: 'sunny',
 57: 'at',
 58: 'she',
 59: 'picture',
 60: 'be',
 61: 'but',
 62: 'his',
 63: 'brown',
 64: 'just',
 65: 'day',
 66: 'that',
 67: 'blue',
 68: 'an',
 69: 'think',
 70: 'around',
 71: 'red',
 72: 'woman',
 73: 'sitting',
 74: 'room',
 75: 'kind',
 76: 'visible',
 77: 'some',
 78: 'water',
 79: 'person',
 80

In [14]:
# INITIALIZE EMBEDDINGS TO RANDOM VALUES
embed_size = 300
vocab_size = len(id2word)
sd = 1/np.sqrt(embed_size)  # Standard deviation to use

weights = np.random.normal(0, scale=sd, size=[vocab_size, embed_size])
weights = weights.astype(np.float32)

In [17]:
from io import open
file = "/home/quanguet/datasets/glove/glove.840B.300d.txt"

with open(file, encoding='utf-8', mode="r") as textFile:
    for line in textFile:
        line = line.split()
        word = line[0]
        word = ''.join(line[:-300])
        
        idx = word2id.get(word, None)
        if idx is not None:
            weights[idx] = np.array(line[-300:], dtype=np.float32)

In [49]:
torch_weights = torch.from_numpy(weights)

In [50]:
embeddings = torch.nn.Embedding(len(word2id), 300)
embeddings

Embedding(11322, 300)

In [51]:
embeddings.weight.data = torch_weights

In [52]:
PATH = 'embedding_Glove_840_300d.pkl'

In [53]:
torch.save(embeddings.state_dict(), PATH)

In [54]:
w = torch.load(PATH)

In [55]:
w

OrderedDict([('weight',
              tensor([[ 1.0779e-01,  5.2718e-02,  1.0397e-01,  ...,  5.0603e-02,
                       -2.1090e-02, -1.0250e-01],
                      [ 3.7493e-01, -6.5246e-01,  9.1198e-01,  ..., -1.2340e+00,
                        5.6611e-01, -3.8912e-01],
                      [ 2.0382e-02,  6.4951e-02, -1.1143e-03,  ..., -8.6877e-02,
                       -3.1497e-02, -4.2145e-02],
                      ...,
                      [ 1.7951e-01,  2.8668e-01,  1.0168e+00,  ...,  6.2386e-01,
                       -3.0787e-01, -2.5465e-01],
                      [ 1.7875e-01,  8.7482e-03,  6.0947e-01,  ...,  5.6894e-02,
                        5.8610e-02,  3.3425e-02],
                      [ 8.2303e-01, -4.8620e-01,  1.6640e-01,  ...,  1.2718e-01,
                        4.4752e-01, -7.6716e-02]]))])